In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
data = pd.read_csv("C:/Users/zhang/Desktop/NYU/2019 Fall/Humane/Data/HMAHCC_HOLDOUT.csv")

In [20]:
data.head()

,ID,EVENT_DESCR,EVENT_ATTR1,EVENT_ATTR2,EVENT_ATTR3,EVENT_ATTR4,EVENT_ATTR5,EVENT_ATTR6,EVENT_ATTR7,EVENT_ATTR8,EVENT_ATTR9,EVENT_ATTR10,PAY_DAY_SUPPLY_CNT,PAYABLE_QTY,MME,DRUG_TYPE,SPECIALTY,SPECIALTY2,SPECIALTY3,days
0,ID98975196852,RX Claim - Rejected,DENY DUE TO ERRORS,"SYMPTOMS, SIGNS, ILL DEFINED CONDITIONS",NO DUAL COVERAGE,COPAY EXCEPTIONS,AMBIEN,ZOLPIDEM TARTRATE TAB 10 MG,2.45421e+07,180,510.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-964
1,ID98975196852,RX Claim - Rejected,DENY DUE TO ERRORS,"SYMPTOMS, SIGNS, ILL DEFINED CONDITIONS",NO DUAL COVERAGE,COPAY EXCEPTIONS,AMBIEN,ZOLPIDEM TARTRATE TAB 10 MG,2.45421e+07,90,255.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-964
2,ID98975196852,RX Claim - Rejected,PRE-AUTH CLM,"SYMPTOMS, SIGNS, ILL DEFINED CONDITIONS",NO DUAL COVERAGE,COPAY EXCEPTIONS,AMBIEN,ZOLPIDEM TARTRATE TAB 10 MG,2.45421e+07,10,63.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-964
3,ID98975196852,RX Claim - New Drug,492700,PROTON PUMP INHIBITORS,OMEPRAZOLE,006,GASTRO,NaN,NaN,OMEPRAZOLE CAP DELAYED RELEASE 20 MG,7.15,49270060.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-963
4,ID98975196852,RX Claim - Paid,PROTON PUMP INHIBITORS,NaN,7.15,0,OMEPRAZOLE,GASTRO,NaN,OMEPRAZOLE CAP DELAYED RELEASE 20 MG,7.15,49270060.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-963


In [22]:
# Find <id, Days> where 'RX Claim - Paid' or 'RX Claim - New Drug' or 'RX CLaim - First Time Main Order' has the same generic as
# 'RX Claim - Rejected' when 'RX Claim - Rejected' == 'REVERSAL'

# We only care about Opioid rows
sub_data = data[(data['EVENT_ATTR1'] == 'REVERSAL') | (data['MME'].isna() == False)]
sub_data['Generic'] = np.where(sub_data['EVENT_DESCR'].isin(['RX Claim - Rejected']), \
                               sub_data['EVENT_ATTR6'], sub_data['EVENT_ATTR8'])

D:\Downloads\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [23]:
# Only include Opioid generic

opioid_generic = sub_data[sub_data['EVENT_DESCR'] == 'RX Claim - Paid']['Generic'].unique()
sub_data = sub_data[sub_data['Generic'].isin(opioid_generic)]

In [28]:
# Running on Colab

id_day = []
for uid in sub_data['ID'].unique():
    for day in sub_data[sub_data['ID'] == uid]['days'].unique():
        sample = sub_data[(sub_data['ID'] == uid) & (sub_data['days'] == day)]
        if 'REVERSAL' in sample['EVENT_ATTR1'].unique() and sample['EVENT_DESCR'].nunique() >= 2\
        and sample['Generic'].nunique() < sample.shape[0] :
            id_day.append([uid, day])

In [29]:
# ????? need more detail to be explained

for uid, day in id_day:
    for generic in data[(data['ID'] == uid) & (data['days'] == day) & (data['EVENT_ATTR1'] == 'REVERSAL')]['EVENT_ATTR6'].unique():
        if data[(data['ID'] == uid) & (data['days'] == day) & \
                (data['EVENT_DESCR'] == 'RX Claim - Paid') & (data['EVENT_ATTR8'] == generic)].shape[0] > 0:
            
            row_index_paid = data[(data['ID'] == uid) & (data['days'] == day) & \
                                  (data['EVENT_DESCR'] == 'RX Claim - Paid') & (data['EVENT_ATTR8'] == generic)].index
            for i in row_index_paid:
                data.at[row_index_paid, 'PAY_DAY_SUPPLY_CNT'] = 0
                data.at[row_index_paid, 'PAYABLE_QTY'] = 0
                data.at[row_index_paid, 'MME'] = 0
            
        if data[(data['ID'] == uid) & (data['days'] == day) & \
                (data['EVENT_DESCR'] == 'RX Claim - New Drug') & (data['EVENT_ATTR8'] == generic)].shape[0] > 0:
            
            row_index_new = data[(data['ID'] == uid) & (data['days'] == day) & \
                                  (data['EVENT_DESCR'] == 'RX Claim - New Drug') & (data['EVENT_ATTR8'] == generic)].index
            for i in row_index_paid:
                data.at[row_index_new, 'PAY_DAY_SUPPLY_CNT'] = 0
                data.at[row_index_new, 'PAYABLE_QTY'] = 0
                data.at[row_index_new, 'MME'] = 0
            
        if data[(data['ID'] == uid) & (data['days'] == day) & \
                (data['EVENT_DESCR'] == 'RX Claim - First Time Mail Order') & (data['EVENT_ATTR8'] == generic)].shape[0] > 0:
            
            row_index_mail = data[(data['ID'] == uid) & (data['days'] == day) & \
                                 (data['EVENT_DESCR'] == 'RX Claim - First Time Mail Order') & \
                                 (data['EVENT_ATTR8'] == generic)].index
            for i in row_index_paid:
                data.at[row_index_mail, 'PAY_DAY_SUPPLY_CNT'] = 0
                data.at[row_index_mail, 'PAYABLE_QTY'] = 0
                data.at[row_index_mail, 'MME'] = 0

In [30]:
# Find duplicates of "RX Claim Paid", "RX Claim New Drug", 'RX Claim First Time Mail Order' in the same day, if opioid
# Pick the max supply day and change the other as 0

opioid_data = data[data['MME'].isna() == False]
opioid_data = opioid_data[opioid_data.duplicated(['ID', 'EVENT_DESCR', 'days'], keep = False)]
opioid_data_max_supply = opioid_data.sort_values(['ID', 'EVENT_DESCR', 'days', 'PAY_DAY_SUPPLY_CNT'])\
                                    .drop_duplicates(['ID', 'EVENT_DESCR', 'days'], keep = 'last')
change_id = np.setdiff1d(np.array(opioid_data.index), np.array(opioid_data_max_supply.index))
for uid in change_id:
    data.loc[uid, 'PAY_DAY_SUPPLY_CNT'] = 0

In [33]:
feature = data[data['days'] < 0]

In [34]:
# Extract total other diagnosis times, total cost, total paid
# What is not extracted: drug class, brand name, drug description
def RX_claim_paid_other(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'RX Claim - Paid') & (feature['MME'].isna() == True)]
    if instance.shape[0] != 0:
        total_RX_claim_paid_other = instance.shape[0]
        total_RX_claim_paid_cost_other = instance['EVENT_ATTR3'].astype('float32').sum()
        total_RX_claim_paid_paid_other = instance['EVENT_ATTR4'].astype('float32').sum()
        total_RX_claim_paid_responsible_other = instance['EVENT_ATTR9'].astype('float32').sum()
    if instance.shape[0] == 0:    
        total_RX_claim_paid_other = 0
        total_RX_claim_paid_cost_other = 0
        total_RX_claim_paid_paid_other = 0  
        total_RX_claim_paid_responsible_other = 0
        
    return [total_RX_claim_paid_other, total_RX_claim_paid_cost_other, total_RX_claim_paid_paid_other,\
            total_RX_claim_paid_responsible_other]

In [36]:
# Extract total opioid times, total cost, total paid, supply count, quantity, MME
# What is not extracted: brand name, drug description, drug type, Specialty, Speciatly2, Specialty3
def RX_claim_paid_opioid(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'RX Claim - Paid') & (feature['MME'].isna() == False)]
    if instance.shape[0] != 0:
        total_RX_claim_paid_opioid = instance.shape[0]
        total_RX_claim_paid_cost_opioid = instance['EVENT_ATTR3'].astype('float32').sum()
        total_RX_claim_paid_paid_opioid = instance['EVENT_ATTR4'].astype('float32').sum()
        total_RX_claim_paid_responsible_opioid = instance['EVENT_ATTR9'].astype('float32').sum()
        total_RX_claim_paid_supply_opioid = instance['PAY_DAY_SUPPLY_CNT'].sum()
        total_RX_claim_paid_quantity_opioid = instance['PAYABLE_QTY'].sum() 
        total_RX_claim_paid_MME_opioid = instance['MME'].sum()
    if instance.shape[0] == 0:    
        total_RX_claim_paid_opioid = 0
        total_RX_claim_paid_cost_opioid = 0
        total_RX_claim_paid_paid_opioid = 0
        total_RX_claim_paid_responsible_opioid = 0
        total_RX_claim_paid_supply_opioid = 0
        total_RX_claim_paid_quantity_opioid = 0
        total_RX_claim_paid_MME_opioid = 0       
        
    return [total_RX_claim_paid_opioid, total_RX_claim_paid_cost_opioid, total_RX_claim_paid_paid_opioid,\
            total_RX_claim_paid_responsible_opioid, total_RX_claim_paid_supply_opioid, total_RX_claim_paid_quantity_opioid,\
            total_RX_claim_paid_MME_opioid]

In [37]:
# Extract total other new drug times

def RX_claim_new_other(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'RX Claim - New Drug') &\
                       (feature['MME'].isna() == True)]
    if instance.shape[0] != 0:
        total_RX_claim_new_other = instance.shape[0]
        total_RX_claim_new_responsible_other = instance['EVENT_ATTR9'].astype('float32').sum()
    if instance.shape[0] == 0:    
        total_RX_claim_new_other = 0 
        total_RX_claim_new_responsible_other = 0
        
    return [total_RX_claim_new_other, total_RX_claim_new_responsible_other]

In [38]:
# Extract new drug opioid times, total cost, total paid, supply count, quantity, MME

def RX_claim_new_opioid(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'RX Claim - New Drug') &\
                       (feature['MME'].isna() == False)]
    if instance.shape[0] != 0:
        total_RX_claim_new_opioid = instance.shape[0]
        total_RX_claim_new_responsible_opioid = instance['EVENT_ATTR9'].astype('float32').sum()
        total_RX_claim_new_supply_opioid = instance['PAY_DAY_SUPPLY_CNT'].sum()
        total_RX_claim_new_quantity_opioid = instance['PAYABLE_QTY'].sum() 
        total_RX_claim_new_MME_opioid = instance['MME'].sum()
    if instance.shape[0] == 0:    
        total_RX_claim_new_opioid = 0
        total_RX_claim_new_responsible_opioid = 0
        total_RX_claim_new_supply_opioid = 0
        total_RX_claim_new_quantity_opioid = 0
        total_RX_claim_new_MME_opioid = 0    
        
        
    return [total_RX_claim_new_opioid, total_RX_claim_new_responsible_opioid, total_RX_claim_new_supply_opioid,\
            total_RX_claim_new_quantity_opioid, total_RX_claim_new_MME_opioid]

In [39]:
# Extract total other first mail times

def RX_claim_mail_other(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'RX Claim - First Time Mail Order') &\
                       (feature['MME'].isna() == True)]
    if instance.shape[0] != 0:
        total_RX_claim_mail_other = instance.shape[0]
        total_RX_claim_mail_responsible_other = instance['EVENT_ATTR9'].astype('float32').sum()
    if instance.shape[0] == 0:    
        total_RX_claim_mail_other = 0  
        total_RX_claim_mail_responsible_other = 0
    return [total_RX_claim_mail_other, total_RX_claim_mail_responsible_other]

In [59]:
# Extract opioid first mail times, supply count, quantity, MME

def RX_claim_mail_opioid(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'RX Claim - First Time Mail Order') &\
                       (feature['MME'].isna() == False)]
    if instance.shape[0] != 0:
        total_RX_claim_mail_opioid = instance.shape[0]
        total_RX_claim_mail_responsible_opioid = instance['EVENT_ATTR9'].astype('float32').sum()
        total_RX_claim_mail_supply_opioid = instance['PAY_DAY_SUPPLY_CNT'].sum()
        total_RX_claim_mail_quantity_opioid = instance['PAYABLE_QTY'].sum() 
        total_RX_claim_mail_MME_opioid = instance['MME'].sum()
    if instance.shape[0] == 0:    
        total_RX_claim_mail_opioid = 0
        total_RX_claim_mail_responsible_opioid = 0
        total_RX_claim_mail_supply_opioid = 0
        total_RX_claim_mail_quantity_opioid = 0
        total_RX_claim_mail_MME_opioid = 0       
        
    return [total_RX_claim_mail_opioid, total_RX_claim_mail_responsible_opioid, total_RX_claim_mail_supply_opioid,\
            total_RX_claim_mail_quantity_opioid, total_RX_claim_mail_MME_opioid]

In [42]:
# Extract total reject times

def RX_claim_rejected(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'RX Claim - Rejected')]
    if instance.shape[0] != 0:
        total_RX_claim_rejected = instance.shape[0]
        total_RX_claim_rejected_supply = instance['EVENT_ATTR8'].astype('float32').sum()
        total_RX_claim_rejected_responsible = instance['EVENT_ATTR9'].astype('float32').sum()
    if instance.shape[0] == 0:    
        total_RX_claim_rejected = 0  
        total_RX_claim_rejected_supply = 0
        total_RX_claim_rejected_responsible = 0
        
    return [total_RX_claim_rejected, total_RX_claim_rejected_supply, total_RX_claim_rejected_responsible]

In [43]:
# Extract total inbound call by mbr times

def Inbound_call_by_mbr(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'Inbound Call by Mbr')]
    if instance.shape[0] != 0:
        inbound_call_by_mbr = instance.shape[0]
    if instance.shape[0] == 0:    
        inbound_call_by_mbr = 0  
        
    return [inbound_call_by_mbr]

In [44]:
# Extract total inbound call by other times

def Inbound_call_by_other(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'Inbound Call by Other')]
    if instance.shape[0] != 0:
        inbound_call_by_other = instance.shape[0]
    if instance.shape[0] == 0:    
        inbound_call_by_other = 0  
        
    return [inbound_call_by_other]

In [45]:
# Extract total inbound call by prov times

def Inbound_call_by_prov(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'Inbound Call by Prov')]
    if instance.shape[0] != 0:
        inbound_call_by_prov = instance.shape[0]
    if instance.shape[0] == 0:    
        inbound_call_by_prov = 0  
        
    return [inbound_call_by_prov]

In [47]:
# Extract total fully paid claim times, total charge, total net paid, total responsible amount

def Fully_paid_claim(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'Fully Paid Claim')]
    if instance.shape[0] != 0:
        fully_paid_claim = instance.shape[0]
        fully_paid_claim_charge = instance['EVENT_ATTR3'].astype('float32').sum()
        fully_paid_claim_paid = instance['EVENT_ATTR4'].astype('float32').sum()
        fully_paid_claim_responsible = instance['EVENT_ATTR5'].astype('float32').sum()
    if instance.shape[0] == 0:    
        fully_paid_claim = 0
        fully_paid_claim_charge = 0
        fully_paid_claim_paid = 0
        fully_paid_claim_responsible = 0   
        
    return [fully_paid_claim, fully_paid_claim_charge, fully_paid_claim_paid, fully_paid_claim_responsible]

In [49]:
# Extract total New Diagonosis Top 5 claim times, total charge, total net paid, total responsible amount

def Top_five(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'New diagnosis - Top 5')]
    if instance.shape[0] != 0:
        top_five = instance.shape[0]
        top_five_charge = instance['EVENT_ATTR3'].astype('float32').sum()
        top_five_paid = instance['EVENT_ATTR4'].astype('float32').sum()
        top_five_responsible = instance['EVENT_ATTR5'].astype('float32').sum()
    if instance.shape[0] == 0:    
        top_five = 0
        top_five_charge = 0
        top_five_paid = 0
        top_five_responsible = 0   
        
    return [top_five, top_five_charge, top_five_paid, top_five_responsible]

In [50]:
# Extract total New Diagonosis CAD claim times, total charge, total net paid, total responsible amount

def CAD(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'New diagnosis - CAD')]
    if instance.shape[0] != 0:
        cad = instance.shape[0]
        cad_charge = instance['EVENT_ATTR3'].astype('float32').sum()
        cad_paid = instance['EVENT_ATTR4'].astype('float32').sum()
        cad_responsible = instance['EVENT_ATTR5'].astype('float32').sum()
    if instance.shape[0] == 0:    
        cad = 0
        cad_charge = 0
        cad_paid = 0
        cad_responsible = 0   
        
    return [cad, cad_charge, cad_paid, cad_responsible]

In [51]:
# Extract total New Diagonosis Diabetes claim times, total charge, total net paid, total responsible amount

def Diabetes(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'New diagnosis - Diabetes')]
    if instance.shape[0] != 0:
        diabetes = instance.shape[0]
        diabetes_charge = instance['EVENT_ATTR3'].astype('float32').sum()
        diabetes_paid = instance['EVENT_ATTR4'].astype('float32').sum()
        diabetes_responsible = instance['EVENT_ATTR5'].astype('float32').sum()
    if instance.shape[0] == 0:    
        diabetes = 0
        diabetes_charge = 0
        diabetes_paid = 0
        diabetes_responsible = 0   
        
    return [diabetes, diabetes_charge, diabetes_paid, diabetes_responsible]

In [52]:
# Extract total New Diagonosis Hypertension claim times, total charge, total net paid, total responsible amount

def Hypertension(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'New diagnosis - Hypertension')]
    if instance.shape[0] != 0:
        hypertension = instance.shape[0]
        hypertension_charge = instance['EVENT_ATTR3'].astype('float32').sum()
        hypertension_paid = instance['EVENT_ATTR4'].astype('float32').sum()
        hypertension_responsible = instance['EVENT_ATTR5'].astype('float32').sum()
    if instance.shape[0] == 0:    
        hypertension = 0
        hypertension_charge = 0
        hypertension_paid = 0
        hypertension_responsible = 0   
        
    return [hypertension, hypertension_charge, hypertension_paid, hypertension_responsible]

In [53]:
# Extract total New Diagonosis CPD claim times, total charge, total net paid, total responsible amount

def CPD(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'New diagnosis - CPD')]
    if instance.shape[0] != 0:
        cpd = instance.shape[0]
        cpd_charge = instance['EVENT_ATTR3'].astype('float32').sum()
        cpd_paid = instance['EVENT_ATTR4'].astype('float32').sum()
        cpd_responsible = instance['EVENT_ATTR5'].astype('float32').sum()
    if instance.shape[0] == 0:    
        cpd = 0
        cpd_charge = 0
        cpd_paid = 0
        cpd_responsible = 0   
        
    return [cpd, cpd_charge, cpd_paid, cpd_responsible]

In [54]:
# Extract total New Diagonosis CHF claim times, total charge, total net paid, total responsible amount

def CHF(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'New diagnosis - CHF')]
    if instance.shape[0] != 0:
        chf = instance.shape[0]
        chf_charge = instance['EVENT_ATTR3'].astype('float32').sum()
        chf_paid = instance['EVENT_ATTR4'].astype('float32').sum()
        chf_responsible = instance['EVENT_ATTR5'].astype('float32').sum()
    if instance.shape[0] == 0:    
        chf = 0
        chf_charge = 0
        chf_paid = 0
        chf_responsible = 0   
        
    return [chf, chf_charge, chf_paid, chf_responsible]

In [55]:
# Extract total Surgery claim times, total charge, total net paid, total responsible amount

def Surgery(uid):
    instance = feature[(feature['ID'] == uid) & (feature['EVENT_DESCR'] == 'Surgery')]
    if instance.shape[0] != 0:
        surgery = instance.shape[0]
        surgery_charge = instance['EVENT_ATTR3'].astype('float32').sum()
        surgery_paid = instance['EVENT_ATTR4'].astype('float32').sum()
        surgery_responsible = instance['EVENT_ATTR5'].astype('float32').sum()
    if instance.shape[0] == 0:    
        surgery = 0
        surgery_charge = 0
        surgery_paid = 0
        surgery_responsible = 0   
        
    return [surgery, surgery_charge, surgery_paid, surgery_responsible]

In [56]:
columns = ['id', 'total_RX_claim_paid_other', 'total_RX_claim_paid_cost_other', 'total_RX_claim_paid_paid_other', \
           'total_RX_claim_paid_responsible_other',\
           'total_RX_claim_paid_opioid', 'total_RX_claim_paid_cost_opioid', 'total_RX_claim_paid_paid_opioid', \
           'total_RX_claim_paid_responsible_opioid',\
           'total_RX_claim_paid_supply_opioid', 'total_RX_claim_paid_quantity_opioid', 'total_RX_claim_paid_MME_opioid', \
           'total_RX_claim_new_other', 'total_RX_claim_new_responsible_other',\
           'total_RX_claim_new_opioid', 'total_RX_claim_new_responsible_opioid', 'total_RX_claim_new_supply_opioid', \
           'total_RX_claim_new_quantity_opioid', 'total_RX_claim_new_MME_opioid', 'total_RX_claim_mail_other', \
           'total_RX_claim_mail_responsible_other', 'total_RX_claim_mail_opioid', 'total_RX_claim_mail_responsible_opioid',
           'total_RX_claim_mail_supply_opioid', 'total_RX_claim_mail_quantity_opioid', \
           'total_RX_claim_mail_MME_opioid', 'total_RX_claim_reject', \
           'total_RX_claim_rejected_supply', 'total_RX_claim_rejected_responsible',
           'inbound_call_by_mbr', 'inbound_call_by_other', \
           'inbound_call_by_prov', 'fully_paid_claim', 'fully_paid_claim_charge', 'fully_paid_claim_paid', \
           'fully_paid_claim_responsible', 'top_five', 'top_five_charge', 'top_five_paid', 'top_five_responsible', 'cad', \
           'cad_charge', 'cad_paid', 'cad_responsible', 'diabetes', 'diabetes_charge', 'diabetes_paid', 'diabetes_responsible', \
           'hypertension', 'hypertension_charge', 'hypertension_paid', 'hypertension_responsible', 'cpd', 'cpd_charge', \
           'cpd_paid', 'cpd_responsible', 'chf', 'chf_charge', 'chf_paid', 'chf_responsible', 'surgery', 'surgery_charge', \
           'surgery_paid', 'surgery_responsible']

In [57]:
processed_holdout = pd.DataFrame(columns = columns)

In [61]:
for uid in feature['ID'].unique():
    one_row = []
    one_row.append(uid)
    one_row.extend(RX_claim_paid_other(uid))
    one_row.extend(RX_claim_paid_opioid(uid))
    one_row.extend(RX_claim_new_other(uid))
    one_row.extend(RX_claim_new_opioid(uid))
    one_row.extend(RX_claim_mail_other(uid))
    one_row.extend(RX_claim_mail_opioid(uid))
    one_row.extend(RX_claim_rejected(uid))
    one_row.extend(Inbound_call_by_mbr(uid))
    one_row.extend(Inbound_call_by_other(uid))
    one_row.extend(Inbound_call_by_prov(uid))
    one_row.extend(Fully_paid_claim(uid))
    one_row.extend(Top_five(uid))
    one_row.extend(CAD(uid))
    one_row.extend(Diabetes(uid))
    one_row.extend(Hypertension(uid))
    one_row.extend(CPD(uid))
    one_row.extend(CHF(uid))
    one_row.extend(Surgery(uid))
    processed_holdout = processed_holdout.append(pd.Series(one_row, index = processed_holdout.columns), ignore_index = True)

In [ ]:
processed_holdout.to_csv('processed_holdout.csv')